In [ ]:
import boto3
import pandas as pd
import logging

In [ ]:
# open files from s3 bucket and processing them

s3 = boto3.resource('s3')
name_bucket = "bucket-ingestion"
my_bucket = s3.Bucket(name=name_bucket)


# open all files from raw bucket and join them in only one table
count = 0
for my_bucket_object in my_bucket.objects.all():
            
    if count != 0:
    
        path = f"s3://bucket-ingestion/{my_bucket_object.key}"

        df_2 = pd.read_csv(path)
        df = pd.concat([df, df_2]).sort_values(by='id_match')
        count += 1
        
    if count == 0:
        path = f"s3://bucket-ingestion/{my_bucket_object.key}"
        df = pd.read_csv(path)
        count += 1
df = df.reset_index(drop=True)

# creating 3 news columns: win, lose, and draw

win = []
draw = []
lose = []
for i in range (len(df)):
    goals = df.iloc[i,12]
    goals_past = df.iloc[i,13]
    if goals > goals_past:
        win.append(1)
        draw.append(0)
        lose.append(0)
        
    if goals < goals_past:
        win.append(0)
        draw.append(0)
        lose.append(1)
        
    if goals == goals_past:
        win.append(0)
        draw.append(1)
        lose.append(0)
df['win'] = win
df['draw'] = draw
df['lose'] = lose

# this three columns have too many words unnecessary
df['stadium'] = df['stadium'].str.slice(20)
df['attendance'] = df['attendance'].str.slice(20)
df['managers'] = df['managers'].str.slice(8)


# taking date style from "3-Mei-21" to "2021-05-03"
months = ['Jan', 'Feb', 'Mac', 'Apr', 'Mei', 'Jun', 'Jul', 'Ago', 'Sep', 'Okt', 'Nov', 'Des']
number_month = [1,2,3,4,5,6,7,8,9,10,11,12] 
date = []
for i in range(len(months)):
    df['date'] = df['date'].str.replace(f'{months[i]}', f'{number_month[i]}')


for i in range(len(df['date'])):
    date_ = df.loc[i,'date'].split('-')
    if len(date_[0]) == 3:
        day = date_[0][1:]
    if len(date_[0]) == 2:
        day = date_[0]
    if len(date_[1]) == 1:
        month = '0'+date_[1]
    if len(date_[1]) != 1:  
        month = date_[1]
    year = '20'+date_[2]
    
    date.append(f'{year}-{month}-{day}')
    
df['date'] = date
df['edition'] = df['edition'].str.replace('-','/')

# saving the file in parquet or csv
df.to_parquet('s3://bucket-curated/soccer_matches.parquet', index = False, engine = 'fastparquet')
df.to_csv('s3://bucket-curated/soccer_matches.parquet', index = False)